In [ ]:
# !pip install 'sagemaker[local]' sagemaker --upgrade 

In [ ]:
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer, json_serializer


sess = sagemaker.Session()
role = 'arn:aws:iam::<your-sagemaker-role>:role/service-role/AmazonSageMaker-ExecutionRole-20181229T135829'# get_execution_role()
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

In [ ]:
training_data = 's3://autogluon/datasets/Inc/train.csv'
test_data = 's3://autogluon/datasets/Inc/test.csv'

In [ ]:
image = 'autogluon-tabular-data'

sm_model = Estimator(image_name=image, 
                     role=role, 
                     train_instance_count=1, 
                     train_instance_type='local',
                     hyperparameters={'label-column': 'class'}
                    )

In [ ]:
sm_model.fit({'training': training_data, 'testing': test_data})

In [ ]:
predictor = sm_model.deploy(1, 'ml.m4.xlarge', serializer=json_serializer)

In [ ]:
import pandas as pd
import json
import io

In [ ]:
df = pd.read_csv('s3://datafy-data-lake-public-artifacts/Inc/test.csv')

In [ ]:
data = df.head()[['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']]

In [ ]:
record = data.head(1).T.to_dict()[0]

payload = json.dumps(record)

In [ ]:
d = predictor.predict(payload)

In [ ]:
predictor.delete_endpoint()